# Oblivious Amplitude Amplification

Here, we implement the "oblivious" version of Grover's algorithm. We suppose $|a\rangle$ and $|b\rangle$ are two kets in a possibly large Hilbert space $\mathcal{H}$. We are given access to the unitaries $e^{i\phi}|a\rangle\langle a|$, $e^{i\phi}|b\rangle\langle b|$ for arbitrary $\phi$, as well as a fixed unitary $U$ for which all we know is that $\langle a|U|b\rangle \neq 0$. The aim of the algorithm is to construct, using these resources, a unitary $Q$ such that $|\langle a|Q|b\rangle| \approx 1$.

For our purposes, we consider a system of six qubits where $|a\rangle$ is a superposition of all strings with random nonzero weights, and $|b\rangle$ is the particular string $|111111\rangle$.

In [38]:
'''Some generic python and jupyter imports'''
import numpy as np
from numpy import pi,sqrt

'''Qiskit imports'''
from qiskit import(
    QuantumCircuit
    , execute
    , Aer
    , ClassicalRegister
    , QuantumRegister
)
backend_svec = Aer.get_backend('statevector_simulator')
backend_qasm = Aer.get_backend('qasm_simulator')

'''Visualizations'''
from qiskit.visualization import plot_histogram
import matplotlib.pyplot as plt

### 1. State Preparation
First we begin by preparing our states. We are given that $|a\rangle$ is a superposition of all strings with random nonzero weights, and $|b\rangle$ is the particular string $|111111\rangle$.

In [54]:
# define qubits
num_qubits = 6
qc = QuantumCircuit(num_qubits)

# |b> = |111111>
qc.x(range(num_qubits)) # initialize b

# |a>: superposition state with random weights
random_weights = np.random.rand(2**num_qubits)
random_weights /= np.linalg.norm(random_weights)
state_vector_a = random_weights
qc_a = qc.initialize(state_vector_a, range(num_qubits))


### 2. Define Reflection Operators
We now define our reflection operators using $e^{i\phi}|a\rangle\langle a|$ and $e^{i\phi}|b\rangle\langle b|$, using $\phi = \pi$.

$R_a = I - 2 |a\rangle\langle a|$

$R_b = I - 2 |b\rangle\langle b|$

In [ ]:
phi = np.pi

# R_a = I - 2|a><a|
dim = len(state_vector_a)
projector = np.outer(state_vector_a, state_vector_a.conj()) # |a><a|
R_a = np.eye(dim) - 2*projector # R_a = I - 2|a><a|

# R_b = I - 2|b><b|
R_b_qc = QuantumCircuit(num_qubits)
mcx = QuantumCircuit.MCXGate(num_qubits-1)
R_b_qc.append(mcx, range(num_qubits))

### 3. Define Reflection Operators